In [1]:
%AddDeps org.sameersingh.scalaplot scalaplot 0.0.4
%AddDeps com.google.protobuf protobuf-java 2.6.1
%AddDeps com.databricks spark-csv_2.10 1.5.0 --transitive

Marking org.sameersingh.scalaplot:scalaplot:0.0.4 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps3630819211546819142/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps3630819211546819142/https/repo1.maven.org/maven2/org/sameersingh/scalaplot/scalaplot/0.0.4/scalaplot-0.0.4.jar
Marking com.google.protobuf:protobuf-java:2.6.1 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps3630819211546819142/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps3630819211546819142/https/repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
Marking com.databricks:spark-csv_2.10:1.5.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps3630819211546819142/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps3630819211546819142/https/repo1.maven.org/maven2/com/univocity/univocity-parsers/1.5.1/univocity-parsers-1.5.1.jar
-> New file at /tmp/toree_add_deps3630819211546819142/htt

In [2]:
val sqlContext = org.apache.spark.sql.SQLContext.getOrCreate(sc)

In [3]:
import sqlContext.implicits._
import org.apache.spark.sql.functions._

### Preconditions
- Make sure data has been downloaded with `download_data.py`
- Make sure the first pass of cleaning has been performed with `clean_files.scala`

In [4]:


val files = List("cooking", "crypto", "robotics", "biology", "travel", "diy")

/* Load and print all files */
val df_all = files.map(f => {
                        sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("../dat/"+f+"_clean.csv")
                   }).reduce(_ unionAll _).withColumn("tags", split($"tags", " "))

Name: org.apache.hadoop.mapred.InvalidInputException
Message: Input path does not exist: file:/home/evan/Dev/Kaggle/StackExchange/dat/cooking_clean.csv
StackTrace: org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:285)
org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:228)
org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:304)
org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:199)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
scala.Option.getOrElse(Option.scala:120)
org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
scala.Option.getOrElse(Option.scala:120)
org.apache.spark.rdd.RDD.partitions(RDD

In [5]:
df_all.show

Name: Compile Error
Message: lastException: Throwable = null
<console>:38: error: not found: value df_all
              df_all.show
              ^
StackTrace: 

In [6]:
df_all.count

Name: Compile Error
Message: <console>:38: error: not found: value df_all
              df_all.count
              ^
StackTrace: 

In [14]:
import sqlContext.implicits._
val rdd = df_all.select("tags").rdd.map(_.get(0).toString.split(" "))

In [15]:
val tags = rdd.collect.flatMap(_.map(t => (t, t.split("-").size)))
val tagsDF = tags.toSeq.distinct.toDF("tag", "length")

tags.maxBy(_._2)

(meet-in-the-middle-attack),5)

In [16]:
tagsDF.show

+--------------------+------+
|                 tag|length|
+--------------------+------+
|WrappedArray(baking,|     1|
|            cookies,|     1|
|            texture)|     1|
|  WrappedArray(oven,|     1|
|       cooking-time,|     2|
|              bacon)|     1|
|  WrappedArray(eggs)|     1|
|WrappedArray(subs...|     1|
|please-remove-thi...|     4|
|        baking-soda,|     2|
|      baking-powder)|     2|
| WrappedArray(sauce,|     1|
|              pasta,|     1|
|           tomatoes,|     1|
|    italian-cuisine)|     2|
|              herbs,|     1|
|            parsley)|     1|
|WrappedArray(food...|     2|
|               beef,|     1|
|       cooking-time)|     2|
+--------------------+------+
only showing top 20 rows



### TODO: load all data and do a histogram over tag lengths

In [17]:
tagsDF.sort(-$"length").show(false)

+---------------------------------------+------+
|tag                                    |length|
+---------------------------------------+------+
|meet-in-the-middle-attack)             |5     |
|WrappedArray(meet-in-the-middle-attack)|5     |
|WrappedArray(meet-in-the-middle-attack,|5     |
|WrappedArray(us-visa-waiver-program)   |4     |
|ho-chi-minh-city,                      |4     |
|proof-provenance-of-funds)             |4     |
|us-visa-waiver-program,                |4     |
|change-purpose-of-travel,              |4     |
|t-and-t-citizens)                      |4     |
|WrappedArray(change-purpose-of-travel, |4     |
|gondolas-and-cable-cars)               |4     |
|proof-of-onward-travel,                |4     |
|WrappedArray(us-visa-waiver-program,   |4     |
|ho-chi-minh-city)                      |4     |
|90-180-visa-rules)                     |4     |
|us-visa-waiver-program)                |4     |
|great-wall-of-china)                   |4     |
|90-180-visa-rules, 

In [18]:
import org.sameersingh.scalaplot.Implicits._
val data = tags.map(_._2).groupBy(identity).mapValues(_.length)
val (x, y) = data.toSeq.sortBy(_._1).map(x => (x._1.toDouble, x._2.toDouble)).unzip

In [19]:
kernel.magics.html(output(SVG, barChart(x -> y)))

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.0 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 20000 
 
 
 
 
 40000 
 
 
 
 
 60000 
 
 
 
 
 80000 
 
 
 
 
 100000 
 
 
 
 
 120000 
 
 
 
 
 140000 
 
 
 
 
 160000 
 
 
 
 
 180000 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Label 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Label